In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import networkx as nx
import freeman as fm

In [2]:
g = fm.load('marvel.gml')
g.label_nodes('name')
g.set_all_nodes(size=15, labpos='hover')
g.set_all_edges(color=(0, 0, 0, 0.5))
g.draw()

In [3]:
bc = nx.betweenness_centrality(g)

In [4]:
import pandas as pd
data = pd.DataFrame({
    'id' : [i for i in g.nodes],
    'Name': [g.nodes[i]['name'] for i in g.nodes],
    'ConnAmnt': [g.nodes[i]['amount'] for i in g.nodes],
    'Intermediation (r)': [i for i in bc.values()],
})

In [5]:
char_infos = pd.read_csv('../dataset/characters_stats.csv', index_col=0)
semi_df = pd.merge(data, char_infos, how='inner', on=['Name']) 

In [6]:
import numpy as np
semi_df = semi_df.replace('nan', np.nan)
semi_df = semi_df[semi_df['ConnAmnt'].notna()]
semi_df['ConnAmnt'] = pd.to_numeric(semi_df['ConnAmnt'])
semi_df = semi_df.drop_duplicates(subset=['Name'], keep='first')

In [7]:
# Existem muitos personagens nesses dados cuja soma Total é composta de 
# Intelligence  Strength    Speed    Durability   Power   Combat  Total
#     1            1          1         1           0       1       5
semi_df = semi_df[semi_df['Total'] != 5]

In [8]:
import statsmodels.api as sm

model = sm.OLS(semi_df['Intermediation (r)'], semi_df[['Intelligence', 'Strength', 'Power']])
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Intermediation (r)   R-squared (uncentered):                   0.451
Model:                            OLS   Adj. R-squared (uncentered):              0.402
Method:                 Least Squares   F-statistic:                              9.052
Date:                Mon, 23 Nov 2020   Prob (F-statistic):                    0.000162
Time:                        22:56:48   Log-Likelihood:                          79.580
No. Observations:                  36   AIC:                                     -153.2
Df Residuals:                      33   BIC:                                     -148.4
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intelligence     0.0006      0.000      3.009      0.005       0.000       0.001
Strength      2.118e-05      0.000      0.131      0.897      -0.000       0.000
Power           -0.0003      0.000     -1.186      0.244      -0.001       0.000
==============================================================================
Omnibus:                        4.598   Durbin-Watson:                   1.118
Prob(Omnibus):                  0.100   Jarque-Bera (JB):                4.267
Skew:                           0.791   Prob(JB):                        0.118
Kurtosis:                       2.413   Cond. No.                         7.25
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
print(f"Coef:\n{result.params}\n")
print(f"std err:\n{result.bse}\n")
print(f"p-values:\n{result.pvalues}\n")

Coef:
Intelligence    0.000552
Strength        0.000021
Power          -0.000287
dtype: float64

std err:
Intelligence    0.000183
Strength        0.000162
Power           0.000242
dtype: float64

p-values:
Intelligence    0.004993
Strength        0.896500
Power           0.244219
dtype: float64

